Import Respository (Replace the Access Token)

In [ ]:
!git clone https://username:<access_token>@github.com/ganeshh123/cloudGAN.git
%cd /content/cloudGAN/
!ls -a -l

Install Requirements

In [ ]:
!pip3 install -r src/requirements.txt

Setup

In [ ]:
#Create env file
import os
import zipfile
import sys
from pathlib import Path

os.system('git update-index --skip-worktree .env')
open('.env', 'a+').close()

userDataFolder = Path(sys.path[0]).resolve() / 'userdata'
# Create User Data Folder
userDataFolder.mkdir(exist_ok=True)
(userDataFolder / 'product_store').mkdir(exist_ok=True)
(userDataFolder / 'rgb_store').mkdir(exist_ok=True)
(userDataFolder / 'mask_store').mkdir(exist_ok=True)
(userDataFolder / 'cyclegan_input').mkdir(exist_ok=True)
(userDataFolder / 'cyclegan_output').mkdir(exist_ok=True)
(userDataFolder / 'cyclegan_output').mkdir(exist_ok=True)

# Absolute Path to Static Data
staticDataPath = open(userDataFolder / 'staticDataPath.txt' , 'w+')
staticDataPath.write(str(Path(sys.path[0]).resolve() / 'data'))
staticDataPath.close()
# Absolute Path to UserData
userDataPath = open(userDataFolder / 'userDataPath.txt' , 'w+')
userDataPath.write(str(userDataFolder.resolve()))
userDataPath.close()

# Create File to Store Parameters
open(userDataFolder / 'parameters.txt' , 'a+').close()
# Create File to Store Search Results for Download
open(userDataFolder / 'searchResults.txt' , 'a+').close()

# Create Output Directories
outputFolder = Path(sys.path[0]).resolve() / 'output'
outputFolder.mkdir(exist_ok=True)
(outputFolder / 'surface').mkdir(exist_ok=True)
(outputFolder / 'masks').mkdir(exist_ok=True)
(outputFolder / 'clouds').mkdir(exist_ok=True)
(outputFolder / 'images').mkdir(exist_ok=True)

print('Unzipping Cloudless Set...')
zipfile.ZipFile(Path(sys.path[0]).resolve() / 'data'/ 'surface_set.zip').extractall((outputFolder / 'surface').resolve())

Enter Copernicus Credentials (Fill Out & Run Cell)

In [ ]:
%%writefile ./.env
# Fill with copernicus username
copernicus_username=
# Fill with copernicus username
copernicus_pass=

# Example
# copernicus_username=myusername
# copernicus_pass=mypass

In [ ]:
Download Products (Optional) - Enter value after productCount

In [ ]:
!python3 src/search_for_cloudy.py --productCount 
!python3 src/copernicus_download.py

try:
    os.remove(userDataFolder / 'imagePaths.txt')
except:
    print('Error Occured Deleting Image Paths File')

Process Inputs for CycleGAN

In [ ]:
try:
    os.remove(userDataFolder / 'imagePaths.txt')
except:
    print('Error Occured Deleting Image Paths File')

!python3 src/unzip_images.py

!python3 src/build_rgb_images.py

!python3 src/build_mask_images.py

!python3 src/slice_images.py

!python3 src/sort_tiles.py

!python3 src/colab_prep_cyclegan.py

Run CycleGAN (GPU Mode)

In [ ]:
!python3 ./src/cyclegan/test.py --dataroot ./userdata/cyclegan_input --name mask-cloud-networks --model cycle_gan --checkpoints_dir ./data --results_dir ./userdata/cyclegan_output --num_test 3500 --load_size 200 --crop_size 200

!python3 src/finish_cyclegan.py

Run CycleGAN (in CPU Mode)

In [ ]:
!python3 ./src/cyclegan/test.py --dataroot ./userdata/cyclegan_input --name mask-cloud-networks --model cycle_gan --checkpoints_dir ./data --results_dir ./userdata/cyclegan_output --num_test 3500 --load_size 200 --crop_size 200 --gpu_ids -1

!python3 src/finish_cyclegan.py

Overlay Clouds

In [ ]:
!python3 src/overlay_clouds.py

Compress Output for Download

In [ ]:
import shutil
print('\n Compressing Output Folder...\n')
shutil.make_archive('./output/training-set', 'zip', './output')
print('\nDone\n')
print('\nFinished - Generated Training Set can be found in outputs folder\n')

Import GDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
Export Produced Training Set Zip to Gdrive

In [ ]:
!cp /content/cloudGAN/output/training-set.zip /content/drive/MyDrive/training-set.zip 